In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
%matplotlib inline

In [ ]:
work_dir = '/home/eva/Documents/gbsa-sims/temperature-sims/temp-11/CC-Hex/GLU/'
top_dir = '/home/eva/Documents/structures/structures-obc2-ff96/CC-Hex/3r3k_W/GLU/'

In [ ]:
work_dir_2 = work_dir+'replicate-2/'
work_dir_3 = work_dir+'replicate-3/'

In [ ]:
system_code = 'CC-Hex*-W, all E deprotonated'

In [ ]:
traj = md.load(work_dir+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop')

traj_trunc = md.iterload(work_dir+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop', chunk = 0, skip = 5000)

for tr in traj_trunc:
    traj_short = tr
print(traj_short)

In [ ]:
traj_2 = md.load(work_dir_2+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop')

traj_trunc_2 = md.iterload(work_dir_2+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop', chunk = 0, skip = 5000)

for tr_2 in traj_trunc_2:
    traj_short_2 = tr_2
print(traj_short_2)

In [ ]:
traj_3 = md.load(work_dir_3+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop')
traj_trunc_3 = md.iterload(work_dir_3+'output.dcd', top = top_dir+'3r3k_W_GLU.prmtop', chunk = 0, skip = 5000)

for tr_3 in traj_trunc_3:
    traj_short_3 = tr_3
print(traj_short_3)

In [ ]:
res_obj = traj.topology.residues
res_list=[]
caps_positions =[]
all_residues=[]


for res in res_obj:
        all_residues.append(str(res))
        if res.name == 'ACE' or res.name == 'NHE':
            print('excluding the ACE and NH2 caps')
            caps_positions.append(str(res))
        else:
            res_list.append(str(res))
            
res_idx=[]
for res in res_list:
    res_idx.append(int(res[3:6]))
    
caps_idx=[]
for cap in caps_positions:
    caps_idx.append(int(cap[3:6]))
    
all_residues_idx=[]
for re in all_residues:
    all_residues_idx.append(int(re[3:6]))

In [ ]:
chain_number = int(len(caps_idx)/2)
print('the number of chains in the structure is', chain_number)

ace=caps_idx[0::2]
nh2=caps_idx[1::2]

chain_list=[]
chain=[]
chain_idx = []
all_idx=[]

for i in range(len(ace)):
    chain = all_residues[(ace[i]+1):(nh2[i])]
    chain_list.append(chain)
    chain=[]

for ch in chain_list:
    for re in ch:
        all_idx.append(int(re[3:6]))    

for i in range(len(ace)):
    chain = all_residues_idx[(ace[i]+1):(nh2[i])]
    chain_idx.append(chain)
    chain=[]
    
chain_length = len(chain_idx[0])
print('the length of each chain is', chain_length, 'residues')

In [ ]:
t = np.arange(0.0, 200.0, 0.004)

t_trunc = np.arange(20.0, 200.0, 0.004)

In [ ]:
rmsds_trunc = md.rmsd(traj_short, traj, 0, atom_indices = [a.index for a in traj.topology.atoms if a.is_backbone])

rmsds_trunc_100 = []

for rmsd in rmsds_trunc:
    rmsds_trunc_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

In [ ]:
rmsds_trunc_2 = md.rmsd(traj_short_2, traj_2, 0, atom_indices = [a.index for a in traj_2.topology.atoms if a.is_backbone])

rmsds_trunc_2_100 =[]
for rmsd in rmsds_trunc_2:
    rmsds_trunc_2_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

In [ ]:
rmsds_trunc_3 = md.rmsd(traj_short_3, traj_3, 0, atom_indices = [a.index for a in traj_3.topology.atoms if a.is_backbone])

rmsds_trunc_3_100 = []
for rmsd in rmsds_trunc_3:
    rmsds_trunc_3_100.append(rmsd/(1+np.log(np.sqrt(chain_number*chain_length/100))))

Conversion of all RMSD to Angstroms

In [ ]:
rmsds_trunc_A_100=[]
rmsds_trunc_2_A_100=[]
rmsds_trunc_3_A_100=[]

for i in rmsds_trunc_100:
    rmsds_trunc_A_100.append(10*i)

for i in rmsds_trunc_2_100:
    rmsds_trunc_2_A_100.append(10*i)

for i in rmsds_trunc_3_100:
    rmsds_trunc_3_A_100.append(10*i)

In [ ]:
rmsd_100_avg = []

for i in range(len(rmsds_trunc_A_100)):
    rmsd_100_avg.append((rmsds_trunc_A_100[i]+rmsds_trunc_2_A_100[i]+rmsds_trunc_3_A_100[i])/3)

In [ ]:
len(rmsd_100_avg)

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, rmsds_trunc_A_100, label = 'replicate 1', color = 'lightskyblue', linewidth=0.5)
ax.plot(t_trunc, rmsds_trunc_2_A_100, label = 'replicate 2', color = 'thistle', linewidth=0.5)
ax.plot(t_trunc, rmsds_trunc_3_A_100, label = 'replicate 3', color = 'yellowgreen', linewidth=0.5)
ax.plot(t_trunc, rmsd_100_avg, label = 'mean', color = 'black', linewidth=1.2)

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ ($\AA$)', title=system_code)
#ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
#ax.set_ylim([0.0, 10.0])

plt.gcf().set_size_inches(10,5)

fig.savefig(work_dir+'analysis/rmsd-trunc-100-mean.png', bbox_inches="tight")

plt.show()

Saving average RMSD to text file

In [ ]:
f = open(work_dir+'rmsd_100_avg.txt', 'w')
for i in rmsd_100_avg:
    f.write(str(i)+'\n')
f.close()

In [ ]:
np.mean(rmsd_100_avg)

In [ ]:
rmsd_100_tuplelist = []

for i in range(len(rmsds_trunc_A_100)):
    rmsd_100_tuplelist.append((rmsds_trunc_A_100[i],rmsds_trunc_2_A_100[i],rmsds_trunc_3_A_100[i]))

In [ ]:
mean_list = []
std_list = []

for tupl in rmsd_100_tuplelist:
    mean_list.append(np.mean(tupl))
    std_list.append(np.std(tupl))

In [ ]:
len(std_list)

In [ ]:
fig,ax = plt.subplots()

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ ($\AA$)', title=system_code)
#ax.grid()

ax.errorbar(t_trunc, mean_list, yerr = std_list, fmt = 'none', ecolor='lightgrey',elinewidth = 1, capsize = 1)
ax.plot(t_trunc, mean_list, color = 'black', linewidth=1.2)

#ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
#ax.set_ylim([0.0, 10.0])

plt.gcf().set_size_inches(10,5)

fig.savefig(work_dir+'analysis/rmsd-trunc-100-with-error.png', bbox_inches="tight")

plt.show()

Comparison between different temperatures

In [ ]:
temp5=[]
temp15=[]
temp25=[]
temp35=[]
temp45=[]
temp55=[]
temp65=[]
temp75=[]
temp85=[]

with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-1/CC-Hex/GLU/rmsd_100_avg.txt') as f:
    for line in f:
        line=line.rstrip()
        temp5.append(float(line))
        
temp15 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-3/CC-Hex/GLU/rmsd_100_avg.txt') as f15:
    for line in f15:
        line=line.rstrip()
        temp15.append(float(line))
        
temp25 = []
with open('/home/eva/Documents/gbsa-sims/sims-3/CC-Hex/3r3k_W/GLU/rmsd_100_avg.txt') as f25:
    for line in f25:
        line=line.rstrip()
        temp25.append(float(line))
        
temp35 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-6/CC-Hex/GLU/rmsd_100_avg.txt') as f35:
    for line in f35:
        line=line.rstrip()
        temp35.append(float(line))
        
temp45 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-7/CC-Hex/GLU/rmsd_100_avg.txt') as f45:
    for line in f45:
        line=line.rstrip()
        temp45.append(float(line))
        
temp55 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-8/CC-Hex/GLU/rmsd_100_avg.txt') as f55:
    for line in f55:
        line=line.rstrip()
        temp55.append(float(line))
        
temp65 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-9/CC-Hex/GLU/rmsd_100_avg.txt') as f65:
    for line in f65:
        line=line.rstrip()
        temp65.append(float(line))
        
temp75 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-10/CC-Hex/GLU/rmsd_100_avg.txt') as f75:
    for line in f75:
        line=line.rstrip()
        temp75.append(float(line))
        
temp85 = []
with open('/home/eva/Documents/gbsa-sims/temperature-sims/temp-11/CC-Hex/GLU/rmsd_100_avg.txt') as f85:
    for line in f85:
        line=line.rstrip()
        temp85.append(float(line))

In [ ]:
fig,ax = plt.subplots()
ax.plot(t_trunc, temp5, label = '5$^\circ$C', color = 'lightcoral', linewidth=1.0)
ax.plot(t_trunc, temp15, label = '15$^\circ$C', color = 'navajowhite', linewidth=1.0)
ax.plot(t_trunc, temp25, label = '25$^\circ$C', color = 'lightgreen', linewidth=1.0)
ax.plot(t_trunc, temp35, label = '35$^\circ$C', color = 'darkgrey', linewidth=1.0)
ax.plot(t_trunc, temp45, label = '45$^\circ$C', color = 'mediumorchid', linewidth=1.0)
ax.plot(t_trunc, temp55, label = '55$^\circ$C', color = 'darkturquoise', linewidth=1.0)
ax.plot(t_trunc, temp65, label = '65$^\circ$C', color = 'seagreen', linewidth=1.0)
ax.plot(t_trunc, temp75, label = '75$^\circ$C', color = 'tomato', linewidth=1.0)
ax.plot(t_trunc, temp85, label = '85$^\circ$C', color = 'gold', linewidth=1.0)

ax.set(xlabel='t (ns)', ylabel='$RMSD_{100}$ ($\AA$)', title=system_code)
#ax.grid()
ax.legend(bbox_to_anchor=(1,1), loc='upper left')
ax.set_xlim([0.0, 200.0])
#ax.set_ylim([0.0, 225.0])

plt.gcf().set_size_inches(10,5)

fig.savefig(work_dir+'temperature-comparison.png', bbox_inches="tight")

plt.show()